#### Tradução de texto com DeepL e análise de sentimento e emoção em texto
 - Necessário conta na DeepL com assinatura gratuita para ter acesso a uma API KEY e 500mil caracteres/mês
 - O DeepL é uma opção para outros tradutores de texto com nível gratuitos e pagos, como Azure AI Translator e Google Translate
 - Necessário conta nos Cloud Services da IBM e da Azure para utilização dos recursos AZURE AI e IBM WATSON

In [66]:
from dotenv import load_dotenv
import os
import deepl # metodo 1
import requests # metodo 2
import pandas as pd

# Tratamento dos textos
import string # processing
import re # processing
from nltk.corpus import stopwords # processing
from nltk.tokenize import RegexpTokenizer # processing
import unidecode # processing

# Sentiment Analysis
from azure.core.credentials import AzureKeyCredential
from azure.ai.textanalytics import TextAnalyticsClient

# Emotion Analysis
from ibm_watson import NaturalLanguageUnderstandingV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson.natural_language_understanding_v1 import Features, EmotionOptions

# Carrega as variáveis de ambiente do arquivo .env
load_dotenv()

api_key = os.getenv("DEEPL_KEY")
azure_key = os.getenv("AZURE_KEY")
azure_endpoint=os.getenv("AZURE_ENDPOINT")
ibm_key = os.getenv("IBM_KEY")

##### Utilizando a LIB da DeepL

In [12]:
# Test
translator = deepl.Translator(api_key)

result = translator.translate_text("Olá, mundo!", target_lang="EN-US")
print(result.text)  

Hello, world!


In [47]:
# Function para Dataframe
# Linguas suportadas: https://www.deepl.com/pt-BR/docs-api/glossaries/list-glossary-languages
def TranslateDeepL(text):
    result = result = translator.translate_text(text, \
        # delete a source_lang para detecção automática de idioma
        source_lang="PT", \
        target_lang="EN-US")
    
    return result

print(TranslateDeepL("Olá, mundo, testando!"))

Hello, world, testing!


##### Utilizando API Rest

In [48]:
def ReqDeepLTranslate(text):
    r = requests.post(
                url="https://api-free.deepl.com/v2/translate",
                data={
                    "source_lang": "PT", # deleta esta linha para detecção de idioma
                    "target_lang": "EN-US",
                    "auth_key": api_key,
                    "text": text,
                },
            )

    return r.json()["translations"][0]["text"]

print(ReqDeepLTranslate("A vida é uma só, aproveite!"))

Life is one, enjoy it!


##### Aplicando tradução a um dataframe

In [23]:
ds = pd.read_csv('data/ptbr-order_reviews.csv')
df = pd.DataFrame(ds.sample(200))
df = df[["score","title","opinion"]].dropna()

display(df)

,score,title,opinion
99548,5,Obrigada!!!,Eu adorei as minhas luminária chegaram perfeit...
91331,5,super recomendo,blza
20696,5,Ótima,Ótima... recomendo
28565,5,SUPER RECOMENDO,"COMO SEMPRE,MUITO SATISFEITA;ENTREGA FEITA MUI..."
38959,4,Produto Lindíssimo,"Produto conforme a imagem ilustrativa, super l..."
82697,1,Muito Chateado com atraso,Muito chateado em pagar 25 de frete \ne ao men...
84207,5,Otimo,Ótimo produto
3389,5,Superou as expectativas,Muito bom!!!
59886,3,Bom,E bonzinho
32780,5,recomendo,"Muito bom, apenas uma não está funcionando."


In [17]:
# Tratamento de texto

def TextToLower(text):
    return text.lower()


def RemoveBlank(text):
    return text.strip()


def RemovePunctuation(text):
    punct = string.punctuation
    punct = punct.replace("'","")
    return text.translate(str.maketrans("","",punct))


def RemoveEmoji(text):
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F" # emoticons
        u"\U0001F300-\U0001F5FF" # simbolos
        u"\U0001F680-\U0001F6FF" # transporte 
        u"\U0001F1E0-\U0001F1FF"
        u"\U00002702-\U000027B0" 
        u"\U000024C2-\U0001F251"
                    "]+", flags = re.UNICODE)
    
    return emoji_pattern.sub(r'', text) 


def RemoveHttp(text):
    http = "https?://\S+|www\.\S+"
    pattern = r"({})".format(http)
    return re.sub(pattern, "", text)


regexp = RegexpTokenizer("[\w']+")
def RemoveStopwords(text, stopwords):
    return " ".join(word for word in regexp.tokenize(text) if word.lower() not in stopwords)



In [18]:
stopwords_pt = set(stopwords.words('english'))

def TextNormalizer(text):
    text = unidecode.unidecode(text)
    text = re.sub('\n','', text)
    text = RemoveHttp(text)
    text = RemoveEmoji(text)
    text = RemovePunctuation(text)
    text = RemoveStopwords(text, stopwords_pt)
    text = RemoveBlank(text)   
    text = TextToLower(text)
    
    return text

In [50]:
df['opinion'] = df['opinion'].apply(TextNormalizer)
df['title'] = df['title'].apply(TextNormalizer)
df['normal_text'] = df['title'].str.cat(df['opinion'], sep=' ')
df['translated'] = df['normal_text'].apply(TranslateDeepL) # Pode alternar entre métodos aqui, porém a lib é bem mais performática

display(df)

,score,title,opinion,normal_text,translated
99548,5,obrigada,eu adorei minhas luminaria chegaram perfeitas ...,obrigada eu adorei minhas luminaria chegaram p...,thank you i love my lamps they arrived perfect...
91331,5,super recomendo,blza,super recomendo blza,I highly recommend it
20696,5,otima,otima recomendo,otima otima recomendo,great great recommend
28565,5,super recomendo,como sempremuito satisfeitaentrega feita muito...,super recomendo como sempremuito satisfeitaent...,super recommendable as alwaysvery satisfieddel...
38959,4,produto lindissimo,produto conforme imagem ilustrativa super lindo,produto lindissimo produto conforme imagem ilu...,beautiful product product as pictured super be...
82697,1,muito chateado com atraso,muito chateado em pagar 25 de frete e ao menos...,muito chateado com atraso muito chateado em pa...,very upset with the delay very upset to pay 25...
84207,5,otimo,otimo produto,otimo otimo produto,great product
3389,5,superou expectativas,muito bom,superou expectativas muito bom,exceeded expectations very good
59886,3,bom,e bonzinho,bom e bonzinho,good and nice
32780,5,recomendo,muito bom apenas uma nao esta funcionando,recomendo muito bom apenas uma nao esta funcio...,I recommend it very highly only one is not wor...


In [51]:
##### A partir daqui pode ser aplicado análise de sentimento e emoção no texto!

##### Análise de Emoção

In [64]:
# config da chave API
authenticator = IAMAuthenticator(ibm_key)

# Cria uma instância do serviço NLU e define a URL
natural_language_understanding = NaturalLanguageUnderstandingV1(
    version='2021-08-01',
    authenticator=authenticator
)
natural_language_understanding.set_service_url('https://api.us-east.natural-language-understanding.watson.cloud.ibm.com/instances/186a3145-a256-4653-945a-cb77e4542307')

In [56]:
# Função
def EmotionAnalysis(text):
    try:
        # Solicitação de análise de emoção
        response = natural_language_understanding.analyze(
            text=text,
            features=Features(emotion=EmotionOptions())
        ).get_result()

        # Extrair emoções
        emocoes = response['emotion']['document']['emotion']
        return max(emocoes, key=emocoes.get)
    except:
        return "insuficiente"

print(EmotionAnalysis("I'm happy today"))

joy


##### Análise de Sentimento

In [67]:
# Essa função funciona melhor com o texto em português, mas nada impede de fazer em inglês e ver os resultados
def SentimentAnalysis(text):
    endpoint = azure_endpoint
    key = azure_key

    text_analytics_client = TextAnalyticsClient(endpoint=endpoint, credential=AzureKeyCredential(key))

    result = text_analytics_client.analyze_sentiment([text], show_opinion_mining=True)
    
    sentiment = result[0]['sentiment']
   
    return sentiment 

print(SentimentAnalysis("Teste"))

neutral


##### Aplicando

In [60]:
df['emotion'] = df['translated'].apply(EmotionAnalysis)
df['sentiment'] = df['normal_text'].apply(SentimentAnalysis)

In [61]:
display(df)

,score,title,opinion,normal_text,translated,emotion,sentiment
99548,5,obrigada,eu adorei minhas luminaria chegaram perfeitas ...,obrigada eu adorei minhas luminaria chegaram p...,thank you i love my lamps they arrived perfect...,joy,positive
91331,5,super recomendo,blza,super recomendo blza,I highly recommend it,joy,positive
20696,5,otima,otima recomendo,otima otima recomendo,great great recommend,joy,positive
28565,5,super recomendo,como sempremuito satisfeitaentrega feita muito...,super recomendo como sempremuito satisfeitaent...,super recommendable as alwaysvery satisfieddel...,joy,positive
38959,4,produto lindissimo,produto conforme imagem ilustrativa super lindo,produto lindissimo produto conforme imagem ilu...,beautiful product product as pictured super be...,joy,positive
82697,1,muito chateado com atraso,muito chateado em pagar 25 de frete e ao menos...,muito chateado com atraso muito chateado em pa...,very upset with the delay very upset to pay 25...,anger,negative
84207,5,otimo,otimo produto,otimo otimo produto,great product,joy,positive
3389,5,superou expectativas,muito bom,superou expectativas muito bom,exceeded expectations very good,sadness,positive
59886,3,bom,e bonzinho,bom e bonzinho,good and nice,insuficiente,positive
32780,5,recomendo,muito bom apenas uma nao esta funcionando,recomendo muito bom apenas uma nao esta funcio...,I recommend it very highly only one is not wor...,joy,positive
